# Exercises

1. Implement a program that iterates through a LunaDataset instance and time how long it takes to do so. In the interest of time it may make sense to have an option to limit the iterations to the first N=1000 samples.

a) How long does it take to run the first time

b) How long does it take to run the second time

c) What does clearing the cache do to the run time

d) What does using the last N=1000 samples do to the first/second runtime?



## 1.a, 1.b, 1.c

In [4]:
from p2ch10.dsets import LunaDataset
import functools

In [5]:
entire_dataset = LunaDataset()
entire_dataset

2022-12-29 17:42:10,753 INFO     pid:59277 p2ch10.dsets:170:__init__ <p2ch10.dsets.LunaDataset object at 0x7f031eee7a00>: 384840 training samples


In [6]:
%%time
for index, (nodule_tens, nodule_label, nodule_series_uid, nodule_center) in enumerate(entire_dataset):
    if index % 100 == 0:
        print(nodule_tens.shape)
    if index == 1000:
        break

torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])


MetaImage: M_ReadElementsData: data not read completely
   ideal = 282066944 : actual = 189792256


torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
CPU times: user 2min 4s, sys: 6min 7s, total: 8min 11s
Wall time: 5min 43s


In [7]:
%%time
for index, (nodule_tens, nodule_label, nodule_series_uid, nodule_center) in enumerate(entire_dataset):
    if index % 100 == 0:
        print(nodule_tens.shape)
    if index == 1000:
        break

torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
torch.Size([1, 32, 48, 48])
CPU times: user 283 ms, sys: 109 ms, total: 393 ms
Wall time: 688 ms


So clearly the cache is being visited here. We should dive in to how lru_cache works

## Aside: caching

In [8]:
@functools.lru_cache
def count_vowels(sentence):
    return sum(sentence.count(vowel) for vowel in 'AEIOUaeiou')

In [9]:
count_vowels.cache_info()

CacheInfo(hits=0, misses=0, maxsize=128, currsize=0)

In [10]:
test_sentence = "Hello World"

In [11]:
count_vowels(test_sentence)

3

In [12]:
count_vowels.cache_info()

CacheInfo(hits=0, misses=1, maxsize=128, currsize=1)

In [13]:
count_vowels(test_sentence)

3

In [14]:
count_vowels.cache_info()

CacheInfo(hits=1, misses=1, maxsize=128, currsize=1)

On further inspection, the project uses a combination of this functools caching and `diskcache`. I suspect the cache created by `functools.lru_cache` is passed to `diskcache.FanoutCache`, which stores the cache in the local filesystem. I guess it must set up some SQLite server which means it isn't simply retrieving it from disc space each time. Although interesting this isn't really the point of what we're doing here. Perhaps one for a later date

## 1.d

The difference here will be that the first 1000 nodules are different from the second 1000 nodules. This is